In [ ]:
#Imports
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt


def cargar_modelo(nombre_archivo):
    with open(nombre_archivo, 'rb') as archivo:
        modelo = pickle.load(archivo)
    return modelo

def contornos_imagen(imagen):
    nombre_modelo = '/modelo_clasificacion_keras.pkl'
    modelo = cargar_modelo(nombre_modelo)
    #gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(imagen, 100, 200)
    plt.imshow(canny)
    plt.show()
    #contornos, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    (contornos,_) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digitos = []
    for c in contornos:
        x, y, w, h = cv2.boundingRect(c)
        if w > 10 and h > 10:
          digito = imagen[y:y+h, x:x+w]
          digitos.append((x, digito))

    digitos.sort(key=lambda x: x[0])  # Ordenar los dígitos de izquierda a derecha

    numero_predicho = ""
    for _, digito_img in digitos:
        numero_predicho += str(predecir_numero(digito_img, modelo))

    return numero_predicho

def preprocesar_imagen(imagen):
    if len(imagen.shape) > 2:
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen_redimensionada = cv2.resize(imagen, (28, 28))
    imagen_aplanada = imagen_redimensionada.flatten()

    # Mostrar la imagen Redimensionada
    plt.imshow(imagen_redimensionada, cmap='gray')
    plt.title('Imagen redimensionada')
    plt.axis('off')
    plt.show()

    return imagen_aplanada

def predecir_numero(imagen, modelo):
    imagen_procesada = preprocesar_imagen(imagen)
    # Realiza la prediccion
    resultado = modelo.predict(np.array([imagen_procesada]))
    print("Resultados de la predicción:", resultado)
    numero_predicho = resultado[0]
    print("Número predicho:", numero_predicho)
    return numero_predicho


def main():
    nombre_modelo = '/modelo_clasificacion_keras.pkl'
    modelo = cargar_modelo(nombre_modelo)

    nombre_imagen = '01.png'
    imagen = cv2.imread(nombre_imagen, cv2.IMREAD_GRAYSCALE)

    if imagen is not None:
        numero_predicho = contornos_imagen(imagen)
        print(f'El número predicho es: {numero_predicho}')
    else:
        print('No se pudo leer la imagen.')

if __name__ == "__main__":
    main()